

[lacentrale](https://www.lacentrale.fr/listing)



# Les voitures selon [lacentrale.fr](https://www.lacentrale.fr/listing)

<img src= ''>

L'objectif de ce projet est de collecter les données des voitures présents sur la page `https://www.lacentrale.fr/listing` du site lacentrale.fr.

Les données que nous collecterons seront les suivantes :
name
price
year
origin
registration_date
technical_inspection
first_hand
mileage
fuel_type
transmission
num_doors
num_seats
power
co2_emission
length
trunk_volume
critair_rating
combined_consumption

In [1]:
pip install --upgrade pip

Note: you may need to restart the kernel to use updated packages.


DEPRECATION: pyodbc 4.0.0-unsupported has a non-standard version number. pip 23.3 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of pyodbc or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063


In [2]:
pip install lxml

Note: you may need to restart the kernel to use updated packages.


DEPRECATION: pyodbc 4.0.0-unsupported has a non-standard version number. pip 23.3 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of pyodbc or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063


In [3]:
pip install selenium

DEPRECATION: pyodbc 4.0.0-unsupported has a non-standard version number. pip 23.3 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of pyodbc or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063


In [1]:

# Le module selenium.webdriver fournit toutes les implémentations de WebDriver : Firefox, Chrome, Edge, ... (sont actuellement prises en charge). 
from selenium import webdriver
from bs4 import BeautifulSoup
import requests

# La classe Keys fournit des touches dans le clavier telles que RETURN, F1, ALT, etc.
from selenium.webdriver.common.keys import Keys
# La classe ActionsChains permet la combinaison de touches, par ex 4 fois la touche Tab 
from selenium.webdriver.common.action_chains import ActionChains

from selenium.webdriver.common.by import By

from datetime import datetime
import time
import random

import pandas as pd
# import pytest

# Lacentrale

In [ ]:
Caractéristiques
Année :
Provenance :
Mise en circulation :
Contrôle technique :
Première main :
Kilométrage compteur :
Energie :
Boite de vitesse :
Nombre de portes :
Puissance :
Emission de CO2 :
Garantie :
    

In [2]:
from lxml import etree
def getLacentralePage(url, page = None):
    if (page != None):
        url = url + '?page=' + page
    response = requests.get(url)
    soup = BeautifulSoup(response.content, features='lxml')

    return soup


def getDetailsCar(carCard):
    car = {
        'nom': None,
        'prix': None,
        'année': None,
        'provenance': None,
        'mise_en_circulation': None,
        'contrôle_technique': None,
        'première_main': None,
        'kilométrage': None,
        'energie': None,
        'boite_vitesse': None,
        'nb_portes': None,
        'puissance': None,
        'emission_CO2': None
    }

    dom = etree.HTML (str(carCard))


    if (carCard.find('div',class_="Text_Text_text SummaryInformation_title__5CYhW Text_Text_headline3") != None):
        car['nom'] = carCard.find('div',class_="Text_Text_text SummaryInformation_title__5CYhW Text_Text_headline3").text.strip()

    if (carCard.find('span',class_="PriceInformation_classifiedPrice__b-Jae") != None):
        car['prix'] = carCard.find('span',class_="PriceInformation_classifiedPrice__b-Jae").text.strip()

    if (len(dom.xpath('//*[@id="year"]/span[2]/span')) > 0):
        car['année'] = dom.xpath('//*[@id="year"]/span[2]/span')[0].text.strip()

    if (len(dom.xpath('//*[@id="origin"]/span[2]/span[1]')) > 0):
        car['provenance'] = dom.xpath('//*[@id="origin"]/span[2]/span[1]')[0].text.strip()

    if (len(dom.xpath('//*[@id="firstCirculationDate"]/span[2]/span')) > 0):
        car['mise_en_circulation'] = dom.xpath('//*[@id="firstCirculationDate"]/span[2]/span')[0].text.strip()

    if (len(dom.xpath('//*[@id="technicalControl"]/span[2]/span[1]')) > 0):
        car['contrôle_technique'] = dom.xpath('//*[@id="technicalControl"]/span[2]/span[1]')[0].text.strip()

    if (len(dom.xpath('//*[@id="firstHand"]/span[2]/span[1]')) > 0):
        car['première_main'] = dom.xpath('//*[@id="firstHand"]/span[2]/span[1]')[0].text.strip()

    if (len(dom.xpath('//*[@id="mileage"]/span[2]/span[1]')) > 0):
        car['kilométrage'] = dom.xpath('//*[@id="mileage"]/span[2]/span[1]')[0].text.strip()

    if (len(dom.xpath('//*[@id="energy"]/span[2]/span')) > 0):
        car['energie'] = dom.xpath('//*[@id="energy"]/span[2]/span')[0].text.strip()

    if (len(dom.xpath('//*[@id="gearbox"]/span[2]/span')) > 0):
        car['boite_vitesse'] = dom.xpath('//*[@id="gearbox"]/span[2]/span')[0].text.strip()

    if (len(dom.xpath('//*[@id="doors"]/span[2]/span')) > 0):
        car['nb_portes'] = dom.xpath('//*[@id="doors"]/span[2]/span')[0].text.strip()

    if (len(dom.xpath('//*[@id="ratedHorsePower"]/span[2]/span[1]')) > 0):
        car['puissance'] = dom.xpath('//*[@id="ratedHorsePower"]/span[2]/span[1]')[0].text.strip()

    if (len(dom.xpath('//*[@id="co2"]/span[2]/div/div/div')) > 0):
        car['emission_CO2'] = dom.xpath('//*[@id="co2"]/span[2]/div/div/div')[0].text.strip()

    return car
    
def getLacentraleCars(soup):
    # Cette fonction est à lancer sur la page avec les voitures à récupérer
    # on récupère toutes les voiture
    # chaque voiture est contenu dans une div avec une classe spécifique
    
    cars = soup.find_all('div', class_=['searchCardContainer', 'boostVo__container'])
    
    data = []
    
    # Pour chaque voiture on récupère les détails qui nous intéressent
    #print(cars)
    for car in cars:
        a = 'https://www.lacentrale.fr/' + car.find('a',class_='Vehiculecard_Vehiculecard_vehiculeCard')['href']
        #print(a)
        page = getLacentralePage(a)
        data.append(getDetailsCar(page))
    
    return data



In [ ]:
# L'URL DU SITE A SCRAPPER
url = "https://www.lacentrale.fr/listing"

# notre liste de voitures
cars = []

# on récupère notre page
homePage = getLacentralePage(url)
maxPage = 300

# on récupère les voitures de toutes les pages

for currentPage in range(1, maxPage+1):
    try:
        print(currentPage)
        page = getLacentralePage(url, str(currentPage))
        data = getLacentraleCars(page)
        [cars.append(car) for car in data]
    except Exception as e:
        print(e)
        pass
df = pd.DataFrame(cars)
df

1
2
3
